In [1]:
!pip install pysentiment2
!pip install pickle5
!pip install transformers

     |████████████████████████████████| 1.9 MB 12.0 MB/s 
     |████████████████████████████████| 256 kB 12.7 MB/s 
     |████████████████████████████████| 4.2 MB 12.1 MB/s 
     |████████████████████████████████| 6.6 MB 45.7 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import tokenize
import re
import torch
from tqdm import tqdm
import random
import pysentiment2 as ps
from google.colab import files

#import 10KQ

unpickled_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/IDX_FILE_TEXT_DAT.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
unpickled_df = unpickled_df[24911:]
unpickled_df

,Unnamed: 0,Unnamed: 0.1,symbol,cik,form,access_number,filed_date,report_url,filing_url,Y_Q,sic,permno,date,text
24911,24911,25700,EOG,821189,10-Q,0000821189-18-000044,2018-11-01,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2018.QTR4,1311,75825,2018-11-01,20181101_10-Q_edgar_data_821189_0000821189-18...
24912,24912,25701,EOG,821189,10-K,0000821189-19-000009,2019-02-26,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR1,1311,75825,2019-02-26,20190226_10-K_edgar_data_821189_0000821189-19...
24913,24913,25702,EOG,821189,10-Q,0000821189-19-000020,2019-05-02,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR2,1311,75825,2019-05-02,20190502_10-Q_edgar_data_821189_0000821189-19...
24914,24914,25703,EOG,821189,10-Q,0000821189-19-000030,2019-08-01,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR3,1311,75825,2019-08-01,20190801_10-Q_edgar_data_821189_0000821189-19...
24915,24915,25704,EOG,821189,10-Q,0000821189-19-000039,2019-11-06,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR4,1311,75825,2019-11-06,20191106_10-Q_edgar_data_821189_0000821189-19...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37357,37360,40061,POOL,945841,10-Q,0000945841-21-000141,2021-07-29,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR3,5090,82515,2021-07-29,20210729_10-Q_edgar_data_945841_0000945841-21...
37358,37361,40062,POOL,945841,10-Q,0000945841-21-000178,2021-10-28,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR4,5090,82515,2021-10-28,20211028_10-Q_edgar_data_945841_0000945841-21...
37359,37362,40063,APA,1841666,10-Q,0001673379-21-000012,2021-05-07,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR2,1311,39490,2021-05-07,20210507_10-Q_edgar_data_1841666_0001673379-2...
37360,37363,40064,APA,1841666,10-Q,0001871133-21-000007,2021-08-05,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR3,1311,39490,2021-08-05,20210805_10-Q_edgar_data_1841666_0001871133-2...


In [6]:
unpickled_df = unpickled_df.reset_index(drop = True)

In [7]:
unpickled_df["negative"] = np.nan
unpickled_df["neutral"] = np.nan
unpickled_df["positive"] = np.nan

unpickled_df['Positive'] = np.nan
unpickled_df["Negative"] = np.nan
unpickled_df['Polarity'] = np.nan
unpickled_df['Subjectivity'] = np.nan

In [8]:
unpickled_df

,Unnamed: 0,Unnamed: 0.1,symbol,cik,form,access_number,filed_date,report_url,filing_url,Y_Q,...,permno,date,text,negative,neutral,positive,Positive,Negative,Polarity,Subjectivity
0,24911,25700,EOG,821189,10-Q,0000821189-18-000044,2018-11-01,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2018.QTR4,...,75825,2018-11-01,20181101_10-Q_edgar_data_821189_0000821189-18...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24912,25701,EOG,821189,10-K,0000821189-19-000009,2019-02-26,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR1,...,75825,2019-02-26,20190226_10-K_edgar_data_821189_0000821189-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24913,25702,EOG,821189,10-Q,0000821189-19-000020,2019-05-02,https://www.sec.gov/Archives/edgar/data/821189...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR2,...,75825,2019-05-02,20190502_10-Q_edgar_data_821189_0000821189-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24914,25703,EOG,821189,10-Q,0000821189-19-000030,2019-08-01,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR3,...,75825,2019-08-01,20190801_10-Q_edgar_data_821189_0000821189-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24915,25704,EOG,821189,10-Q,0000821189-19-000039,2019-11-06,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/821189...,2019.QTR4,...,75825,2019-11-06,20191106_10-Q_edgar_data_821189_0000821189-19...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12446,37360,40061,POOL,945841,10-Q,0000945841-21-000141,2021-07-29,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR3,...,82515,2021-07-29,20210729_10-Q_edgar_data_945841_0000945841-21...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12447,37361,40062,POOL,945841,10-Q,0000945841-21-000178,2021-10-28,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/945841...,2021.QTR4,...,82515,2021-10-28,20211028_10-Q_edgar_data_945841_0000945841-21...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12448,37362,40063,APA,1841666,10-Q,0001673379-21-000012,2021-05-07,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR2,...,39490,2021-05-07,20210507_10-Q_edgar_data_1841666_0001673379-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12449,37363,40064,APA,1841666,10-Q,0001871133-21-000007,2021-08-05,https://www.sec.gov/ix?doc=/Archives/edgar/dat...,https://www.sec.gov/Archives/edgar/data/184166...,2021.QTR3,...,39490,2021-08-05,20210805_10-Q_edgar_data_1841666_0001871133-2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert", model_max_len=512, truncation=True)

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

nlp = pipeline('sentiment-analysis',model=model, tokenizer=tokenizer, device = 0)




Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 23 07:45:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    36W / 250W |   1517MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
p = []

z = unpickled_df.sample(n=2)

for i in tqdm(z['text']):
  try:
    x = tokenize.sent_tokenize(i)
    print(x)

    for j in x:
      p.append(len(j.split()))
  except:
    continue

sent_length_90 = np.percentile(p, 75)

print('The sentence length in the 90th percentile is:'," ",sent_length_90)

100%|██████████| 2/2 [00:00<00:00, 2139.41it/s]

['ITEM 5 OTHER EVENTS On January 27 1999 USX Corporation executed an Underwriting Agreement with J P Morgan Securities Inc Goldman Sachs Co Salomon Smith Barney Inc and PNC Capital Markets Inc in connection with the issuance of 6 65 Notes Due 2006 pursuant to a shelf registration statement on Form S 3 File No 333 56867 ITEM 7 FINANCIAL STATEMENTS AND EXHIBITS c Exhibits 1 Underwriting and Pricing Agreements dated January 27 1999 4 Establishment Action of the Terms and Rate Committees']
['Item 5 Other Events On August 3 2004 MBNA Europe Bank Limited a wholly owned foreign bank subsidiary of MBNA America Bank N A located in the United Kingdom completed the securitization of 125 0 million seven year credit card asset backed notes issued through Chester Asset Receivables Dealings Issuer Limited CARDS MBNA America Bank N A is a wholly owned subsidiary of MBNA Corporation The transaction Series 2004 B1 had one class of publicly traded asset backed notes 125 0 million Class B floating rate as

In [ ]:
unpickled_df.head(50)

,Unnamed: 0,symbol,cik,form,access_number,filed_date,accepted_date,report_url,filing_url,Y_Q,text,negative,neutral,positive,Positive,Negative,Polarity,Subjectivity
0,0,MRO,101778,8-K,0000101778-96-000001,1996-01-26,1996-01-26 00:00:00,https://www.sec.gov/Archives/edgar/data/101778...,https://www.sec.gov/Archives/edgar/data/101778...,1996.QTR1,"Item 5. Other Events\n\nOn January 25, 1996,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,WY,106535,8-K,0000106535-96-000002,1996-02-14,1996-02-14 00:00:00,https://www.sec.gov/Archives/edgar/data/106535...,https://www.sec.gov/Archives/edgar/data/106535...,1996.QTR1,"Item 5. Other Events\n\nOn February 13, 1996,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,JPM,19617,8-K,0000019617-96-000003,1996-01-12,1996-01-12 00:00:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,1996.QTR1,"Items 5. Other Events\n\n\nOn January 5, 1996...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,JPM,19617,8-K,0000019617-96-000006,1996-01-18,1996-01-18 00:00:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,1996.QTR1,Item 5. Other Events\n- ---------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,JPM,19617,8-K,0000019617-96-000009,1996-02-05,1996-02-05 00:00:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,1996.QTR1,Items 5. Other Events\n\nChemical Banking Cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,JPM,19617,8-K,0000019617-96-000043,1996-03-25,1996-03-25 00:00:00,https://www.sec.gov/Archives/edgar/data/19617/...,https://www.sec.gov/Archives/edgar/data/19617/...,1996.QTR1,Items 5. Other Events\n\n\nChemical Banking C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,JNJ,200406,8-K,0000950123-96-001060,1996-03-08,1996-03-08 00:00:00,https://www.sec.gov/Archives/edgar/data/200406...,https://www.sec.gov/Archives/edgar/data/200406...,1996.QTR1,Item 2. Acquisition or Disposition of Assets...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,AMD,2488,8-K,0000898430-96-000094,1996-01-12,1996-01-12 00:00:00,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,1996.QTR1,Item 5. Other Events.\n- ------ -----------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,AMD,2488,8-K,0000898430-96-000095,1996-01-12,1996-01-12 00:00:00,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,1996.QTR1,Item 5. Other Events.\n- ------ -----------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,AMD,2488,8-K,0000898430-96-000096,1996-01-12,1996-01-12 00:00:00,https://www.sec.gov/Archives/edgar/data/2488/0...,https://www.sec.gov/Archives/edgar/data/2488/0...,1996.QTR1,Item 5. Other Events.\n- ------ -----------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
random.seed(3457)

dictio_finbert = {}
dictio_LM = {}

lm = ps.LM()

for i in tqdm(range(len(unpickled_df))):
  
  try:
      y = re.sub(r"[^a-zA-Z0-9 ]", " ", unpickled_df['text'][i])
      y = y.lower()
      y = re.sub(' {2,}', ' ', y)
      
      tokens = lm.tokenize(y)
      a = lm.get_score(tokens)   

      for r in a:
        unpickled_df.loc[i,r] = a[r]
      
      x = tokenize.sent_tokenize(unpickled_df['text'][i])
      
      x = [re.sub(r"[^a-zA-Z0-9 ]", " ", i) for i in x]

      x = [i.lower() for i in x]

      x = [re.sub(' {2,}', ' ', i) for i in x]
      
      x_new = []

      for o in range(len(x)):
          if len(x[o]) <= 1:
              continue
          elif len(x[o].split()) > 60:
              #x_new.append(x[o])
              continue     
          else:
              x_new.append(x[o])     

      z = []
      k = ''
      for p in x_new:
          if (len(tokenizer(k)[0]) + len(tokenizer(p)[0]))<150:
              p = ' ' + p
              k += p
          else:
              k = k[1:]
              if k != '':
                  if k[0] == ' ':
                      k = k[2:]
                      z.append(re.sub(' {2,}', ' ', k))
                      k = ''
                  else:
                      z.append(re.sub(' {2,}', ' ', k))   
                      k = ''
              else:
                  continue
          
      z = random.sample(z, int((len(z)/100)*100))
      
      label_list = []
      score_list = []
      
      for h in z:
          result = nlp(h)
          label = result[0]['label']
          score = result[0]['score']
          label_list.append(label)
          score_list.append(score)
          
      dictio_finbert[i] = [label_list,score_list]
              
      values, counts = np.unique(label_list, return_counts=True)

      ziped = zip(values, counts)

      ziped_list = list(ziped)

      #print(values)   
      #z = counts/sum(counts)
      print(list(ziped))


      if len(ziped_list) > 0:
        for c in ziped_list:

          unpickled_df.loc[i,c[0].lower()] = c[1]

      unpickled_df.loc[i, 'text'] = np.nan
      #print(unpickled_df['report_url'][i])
  except:
      continue


import pickle

pickle_out = open('/content/drive/MyDrive/Colab Notebooks/DATA/10kq_sentiment_new_prosusai_24911_37361.pkl', "wb")
pickle.dump(unpickled_df,pickle_out)
pickle_out.close()


  0%|          | 0/12451 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:998: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
  0%|          | 1/12451 [00:02<7:51:35,  2.27s/it]

[]


  0%|          | 2/12451 [00:08<15:28:20,  4.47s/it]

[]


  0%|          | 3/12451 [00:09<10:34:02,  3.06s/it]

...

[]


  0%|          | 15/12451 [00:34<4:46:09,  1.38s/it]

[]


  0%|          | 16/12451 [00:34<3:57:31,  1.15s/it]

[]


In [ ]:
unpickled_df

NameError: ignored

In [ ]:
output_5 = unpickled_df

output_5

,symbol,cik,form,access_number,filed_date,accepted_date,report_url,filing_url,Y_Q,text,negative,neutral,positive,Positive,Negative,Polarity,Subjectivity
8500,INTC,50863,10-Q,0000050863-14-000058,2014-07-25,2014-07-25 09:02:54,https://www.sec.gov/Archives/edgar/data/50863/...,https://www.sec.gov/Archives/edgar/data/50863/...,2014.QTR3,NaN,17.0,72.0,17.0,184.0,567.0,-0.509987,0.065912
8501,AIG,5272,10-Q,0000005272-14-000010,2014-08-04,2014-08-04 16:04:38,https://www.sec.gov/Archives/edgar/data/5272/0...,https://www.sec.gov/Archives/edgar/data/5272/0...,2014.QTR3,NaN,64.0,218.0,44.0,783.0,2743.0,-0.555871,0.083062
8502,PPL,55387,10-Q,0000922224-14-000091,2014-07-31,2014-07-31 17:28:01,https://www.sec.gov/Archives/edgar/data/55387/...,https://www.sec.gov/Archives/edgar/data/116197...,2014.QTR3,NaN,57.0,907.0,21.0,3146.0,7238.0,-0.394068,0.060912
8503,LLY,59478,10-Q,0000059478-14-000129,2014-07-28,2014-07-28 11:57:12,https://www.sec.gov/Archives/edgar/data/59478/...,https://www.sec.gov/Archives/edgar/data/59478/...,2014.QTR3,NaN,20.0,131.0,14.0,688.0,1632.0,-0.406897,0.081213
8504,L,60086,10-Q,0001193125-14-293769,2014-08-04,2014-08-04 17:21:42,https://www.sec.gov/Archives/edgar/data/60086/...,https://www.sec.gov/Archives/edgar/data/60086/...,2014.QTR3,NaN,31.0,73.0,14.0,251.0,967.0,-0.587849,0.080454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10996,WU,1365135,10-Q,0001365135-18-000045,2018-11-01,2018-11-01 16:11:08,https://www.sec.gov/Archives/edgar/data/136513...,https://www.sec.gov/Archives/edgar/data/136513...,2018.QTR4,NaN,35.0,132.0,4.0,374.0,1345.0,-0.564863,0.088549
10997,V,1403161,10-K,0001403161-18-000055,2018-11-16,2018-11-16 16:10:27,https://www.sec.gov/Archives/edgar/data/140316...,https://www.sec.gov/Archives/edgar/data/140316...,2018.QTR4,NaN,48.0,233.0,29.0,749.0,1545.0,-0.346992,0.070665
10998,BMY,14272,10-Q,0000014272-18-000197,2018-10-25,2018-10-25 11:36:04,https://www.sec.gov/Archives/edgar/data/14272/...,https://www.sec.gov/Archives/edgar/data/14272/...,2018.QTR4,NaN,21.0,74.0,13.0,291.0,528.0,-0.289377,0.070180
10999,ACN,1467373,10-K,0001467373-18-000318,2018-10-24,2018-10-24 17:05:52,https://www.sec.gov/Archives/edgar/data/146737...,https://www.sec.gov/Archives/edgar/data/146737...,2018.QTR4,NaN,63.0,206.0,17.0,912.0,1643.0,-0.286106,0.081797


In [ ]:


import pickle

pickle_out = open('/content/drive/MyDrive/Colab Notebooks/DATA/output_6_11000_13068_10kq_sentiment.pkl', "wb")
pickle.dump(output_6,pickle_out)
pickle_out.close()


In [ ]:
from google.colab import files

unpickled_df.to_pickle('output.pkl') 
files.download('output.pkl')